## Topic: Challenge Set 11
## Subject: Explore Damage Incidents to Ships with Poisson GLMs
## Date: 03/10/2018
## Name: Subramanian Iyer
## Worked With: Worked Individually

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy
from scipy.stats import chi2

/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
reader = pd.io.stata.StataReader('ships.dta')
data = reader.data()
data.head()

/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/pandas/io/stata.py:1412: UserWarning: 'data' is deprecated, use 'read' instead
  warnings.warn("'data' is deprecated, use 'read' instead")


,type,construction,operation,months,damage
0,A,1960-64,1960-74,127.0,0.0
1,A,1960-64,1975-79,63.0,0.0
2,A,1965-70,1960-74,1095.0,3.0
3,A,1965-70,1975-79,1095.0,4.0
4,A,1970-74,1960-74,1512.0,6.0


In [3]:
data.dtypes

type            category
construction    category
operation       category
months           float32
damage           float32
dtype: object

**Challenge Number 1**

In [4]:
y,X = patsy.dmatrices('damage~type+construction+operation+months', data = data, return_type = 'dataframe')
model0 = sm.GLM(y,X,family = sm.families.Poisson(sm.families.links.log))
results0 = model0.fit()
print(results0.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 damage   No. Observations:                   34
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                            9
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -84.182
Date:                Sun, 11 Mar 2018   Deviance:                       70.498
Time:                        17:53:00   Pearson chi2:                     65.8
No. Iterations:                     6                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.1786      0.277      0.645      0.519      -0.364       0.722
type[T.B]    

To my inexperienced eye, that deviance and pearson chi2 look reasonable

**Challenge Number 2**

In [5]:
y2,X2 = patsy.dmatrices('damage~type+construction+operation', data = data, return_type = 'dataframe')
model1 = sm.GLM(y2,X2,family = sm.families.Poisson(sm.families.links.log), offset = np.log(data['months']))
results1 = model1.fit()
print(results1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 damage   No. Observations:                   34
Model:                            GLM   Df Residuals:                       25
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -68.281
Date:                Sun, 11 Mar 2018   Deviance:                       38.695
Time:                        17:53:03   Pearson chi2:                     42.3
No. Iterations:                     6                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -6.4059      0.217    -29.460      0.000      -6.832      -5.980
type[T.B]    

Deviance and chi2 went way down. yay! Yes, this model does perform better.

**Challenge Number 3**

In [6]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test, M_train, M_test = train_test_split(X2, y2, data['months'], test_size = 0.30, \
                                                                     random_state = 4444)
model2 = sm.GLM(y_train,X_train,family = sm.families.Poisson(sm.families.links.log), offset = np.log(M_train))
results2 = model2.fit()
cpreds = model2.predict(params = results2.params, exog = X_test, offset = np.log(M_test))
print('mean squared error: ', end = '')
print(metrics.mean_squared_error(y_test, cpreds))
print('mean absolute error: ', end = '')
print(metrics.mean_absolute_error(y_test, cpreds))

mean squared error: 34.350875385
mean absolute error: 3.79992577276


/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Challenge 4**

In [7]:
model3 = sm.GLM(y2, X2['Intercept'],family = sm.families.Poisson(sm.families.links.log), offset = np.log(data['months']))
results3 = model3.fit()
print(1-chi2.cdf(results3.deviance-results1.deviance, 8))

0.0


With a p value so low the program just returns 0, it's reasonable to say that our model performs better than the null model, and that the extra parameters should be kept.

**Challenge Number 5**

In [8]:
y.head()

,damage
0,0.0
1,0.0
2,3.0
3,4.0
4,6.0


In [9]:
Poispreds = model1.predict(params = results1.params, exog = X2, offset = np.log(data['months']))
linmod = sm.OLS(np.log(y+1),X)
fit = linmod.fit()
Linpreds = np.exp(fit.predict(X))-1
print('ols: ')
print(fit.summary())
print('OLS mean squared error: ')
print(metrics.mean_squared_error(y2, Linpreds))
print(' ')
print('poisson: ')
print(results1.summary())
print('Poisson mean squared error: ')
print(metrics.mean_squared_error(y2, Poispreds))

ols: 
                            OLS Regression Results                            
Dep. Variable:                 damage   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     13.87
Date:                Sun, 11 Mar 2018   Prob (F-statistic):           1.64e-07
Time:                        17:53:13   Log-Likelihood:                -26.076
No. Observations:                  34   AIC:                             72.15
Df Residuals:                      24   BIC:                             87.42
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept           

The coefficients of the two models are nowhere near close, and it would be very surprising if they were.  The poisson model performs better.

**Challenge Number 6**

In [10]:
reader2 = pd.io.stata.StataReader('smoking.dta')
data2 = reader2.data()
data2.head()

/Users/subramanianiyer/anaconda2/envs/mypython3/lib/python3.6/site-packages/pandas/io/stata.py:1412: UserWarning: 'data' is deprecated, use 'read' instead
  warnings.warn("'data' is deprecated, use 'read' instead")


,age,smoke,pop,dead
0,40-44,Doesn't smoke,656.0,18.0
1,45-49,Doesn't smoke,359.0,22.0
2,50-54,Doesn't smoke,249.0,19.0
3,55-59,Doesn't smoke,632.0,55.0
4,60-64,Doesn't smoke,1067.0,117.0


In [11]:
data2.dtypes

age      category
smoke    category
pop       float32
dead      float32
dtype: object

In [12]:
y3,X3 = patsy.dmatrices('dead~age+smoke', data = data2, return_type = 'dataframe')
y4,X4 = patsy.dmatrices('dead~age+smoke+pop', data = data2, return_type = 'dataframe')
modelS = sm.GLM(y3, X3,family = sm.families.Poisson(sm.families.links.log), offset = np.log(data2['pop']))
resultsP = modelS.fit()
modelSL = sm.OLS(y4,X4)
resultsL = modelSL.fit()
print('poisson: ')
print(resultsP.summary())
print('Mean Squared Error: ', end = '')
Ppreds = modelS.predict(params = resultsP.params, exog = X3, offset = np.log(data2['pop']))
print(metrics.mean_squared_error(y3, Ppreds))
print(' ')
print('ols: ')
print(resultsL.summary())
print('Mean Squared Error: ', end = '')
Lpreds = resultsL.predict(X4)
print(metrics.mean_squared_error(y4, Lpreds))

poisson: 
                 Generalized Linear Model Regression Results                  
Dep. Variable:                   dead   No. Observations:                   36
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                           11
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -130.75
Date:                Sun, 11 Mar 2018   Deviance:                       21.487
Time:                        17:53:17   Pearson chi2:                     20.6
No. Iterations:                     6                                         
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -3.68

Sidenote: I tried the last cell with a log transform response variable for the OLS, and it did worse than without the log transform.

Poisson Model performed better.